In [26]:
import pandas as pd
import numpy as np
from siuba import *
from plotnine import *
import plotly.express as px

In [93]:
# Load data for downloaded roads and select ours
df = pd.read_parquet("../data/preprocessed/accidents_from_mapper.parquet")
df_90 = df[df.source_road==90].copy()

In [59]:
df_90 = (
    df
    >> filter(_.source_road==90)
    >> mutate(part=if_else(_.location <= 49.5, 'phase_1', if_else((49.5 > _.location) & (_.location >= 60.4), 'phase_2', 'not_extended')),
        after=_.year >= 2012
    )
    # >> mutate(section=_.part + if_else(_.after, '_after_2012', '_before_2012'))
)
df_90 >> filter(_.location.isna())

,year,month,day of week,day/night,accident_severity,accident_type,killed,severly_injured,lightly_injured,injured_pedestrians,...,localization_quality,settlement,road1,location,road2,road3,road4,source_road,part,after
0,2003,אפריל,שני,לילה,קלה,התנגשות חזית באחור,0.0,0.0,3.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,NaN,NaN,NaN,90,not_extended,False
1,2003,אוקטובר,חמישי,לילה,קלה,התנגשות חזית בצד,0.0,0.0,1.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,9119.0,NaN,NaN,90,not_extended,False
2,2003,אוגוסט,רביעי,לילה,קשה,ירידה מהכביש או עלייה למדרכה,0.0,1.0,1.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,977.0,NaN,NaN,90,not_extended,False
3,2003,יוני,חמישי,לילה,קלה,התנגשות חזית בצד,0.0,0.0,5.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,NaN,NaN,NaN,90,not_extended,False
4,2003,אוגוסט,ראשון,לילה,קלה,התנגשות חזית בחזית,0.0,0.0,11.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,7579.0,NaN,NaN,90,not_extended,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,2022,ינואר,שישי,יום,קלה,התנגשות חזית בצד,0.0,0.0,6.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,91.0,NaN,NaN,90,not_extended,True
2585,2022,ינואר,רביעי,לילה,קלה,התנגשות חזית בצד,0.0,0.0,4.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,8651.0,NaN,NaN,90,not_extended,True
2586,2022,מרס,שישי,לילה,קלה,התנגשות חזית בצד,0.0,0.0,1.0,0.0,...,עיגון מדויק,NaN,85.0,NaN,90.0,NaN,NaN,90,not_extended,True
2587,2022,ינואר,שני,לילה,קלה,התנגשות חזית בצד,0.0,0.0,2.0,0.0,...,עיגון מדויק,NaN,90.0,NaN,9119.0,NaN,NaN,90,not_extended,True


In [28]:
junctions = pd.read_csv("../data/preprocessed/junctions.csv")
junctions

,name,location
0,Eilot,11.9
1,Meches,15.6
2,Beer Ora,27.0
3,Timna,36.0
4,Samar,41.3
5,Yotvata,49.5
6,Grofit,54.0
7,Ktora,60.4
8,Yahel,72.3
9,Menuha,97.6


In [4]:
df

,year,month,day of week,day/night,accident_severity,accident_type,killed,severly_injured,lightly_injured,injured_pedestrians,...,localization_quality,settlement,road1,location,road2,road3,road4,part,after,section
0,2003,אפריל,ראשון,לילה,קשה,התנגשות חזית בצד,0,1,8,0,...,עיגון מדויק,NaN,90,100.3,NaN,NaN,NaN,arava_tihona,False,arava_tihona_before_2012
1,2003,מאי,שישי,לילה,קלה,התהפכות,0,0,3,0,...,עיגון מדויק,NaN,90,75.1,NaN,NaN,NaN,arava_tihona,False,arava_tihona_before_2012
2,2003,מאי,שבת,לילה,קלה,התהפכות,0,0,4,0,...,עיגון מדויק,NaN,90,39.9,NaN,NaN,NaN,arava_dromit,False,arava_dromit_before_2012
3,2003,יוני,חמישי,לילה,קלה,התנגשות חזית באחור,0,0,2,0,...,עיגון מדויק,NaN,90,135.8,NaN,NaN,NaN,arava_tihona,False,arava_tihona_before_2012
4,2003,מרס,שני,לילה,קטלנית,פגיעה בהולך רגל,1,4,22,1,...,עיגון מדויק,NaN,90,43.0,NaN,NaN,NaN,arava_dromit,False,arava_dromit_before_2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2021,אפריל,שלישי,יום,קשה,התהפכות,0,3,0,0,...,עיגון מדויק,NaN,90,131.0,NaN,NaN,NaN,arava_tihona,True,arava_tihona_after_2012
344,2021,אפריל,רביעי,לילה,קטלנית,התנגשות חזית בחזית,2,0,1,0,...,עיגון מדויק,NaN,90,118.5,NaN,NaN,NaN,arava_tihona,True,arava_tihona_after_2012
345,2022,מרס,ראשון,יום,קלה,התנגשות חזית בצד,0,0,4,0,...,עיגון מדויק,NaN,90,NaN,1093.0,NaN,NaN,arava_tihona,True,arava_tihona_after_2012
346,2022,מרס,שבת,לילה,קלה,התנגשות עם עצם דומם,0,0,4,0,...,עיגון מדויק,NaN,90,38.9,NaN,NaN,NaN,arava_dromit,True,arava_dromit_after_2012


In [31]:
casualty_cols = [
    'killed',
    'severly_injured',
    'lightly_injured',
    'injured_pedestrians',
    'casualties_ages_0-19',
    'casualties_ages_20-64',
    'casualties_ages_65_plus',
    'total_casualties',
    'vehicle_count',
    'drivers',
]
summary = df_90.groupby('section')[casualty_cols].sum()
summary

,killed,severly_injured,lightly_injured,injured_pedestrians,casualties_ages_0-19,casualties_ages_20-64,casualties_ages_65_plus,total_casualties,vehicle_count,drivers
section,,,,,,,,,,
arava_dromit_after_2012,9.0,13.0,114.0,2.0,35.0,95.0,4.0,136.0,76.0,76.0
arava_dromit_before_2012,21.0,32.0,320.0,1.0,96.0,268.0,7.0,373.0,181.0,181.0
arava_tihona_after_2012,102.0,319.0,2848.0,38.0,742.0,2224.0,236.0,3269.0,2084.0,2084.0
arava_tihona_before_2012,134.0,439.0,4032.0,45.0,1280.0,3120.0,192.0,4605.0,2542.0,2542.0


In [40]:
df_90.groupby(['part', 'year']).sum().reset_index() >> pipe(lambda x: px.bar(x, x='year', y='killed', color='part', barmode='group'))

In [49]:
(
    df_90
    >> filter(_.part=='arava_dromit', _.year>=2012, _.killed>0, _.location < 49.5)
    >> select('year', 'month', 'day of week', 'day/night', 'accident_type', 'killed', 'location')
    
)

,year,month,day of week,day/night,accident_type,killed,location
2176,2017,אוגוסט,שני,יום,התהפכות,1.0,45.0
2534,2021,יולי,ראשון,לילה,פגיעה בהולך רגל,1.0,31.0


In [6]:
df.value_counts('section')

section
arava_tihona_before_2012    125
arava_dromit_before_2012     89
arava_tihona_after_2012      84
arava_dromit_after_2012      50
dtype: int64

In [7]:
df[casualty_cols].sum()

killed                       87
severly_injured             178
lightly_injured            1099
injured_pedestrians           7
casualties_ages_0-19        379
casualties_ages_20-64       906
casualties_ages_65_plus      65
total_casualties           1364
vehicle_count               629
drivers                     629
dtype: int64

In [8]:
fig = (
    df
    # >> filter(_.part=='south_')
    # >> filter(_.location >= 60.4, _.location <= 180)
    >> group_by(_.section)
    >> count(_.accident_type)
    >> pipe(lambda x: px.bar(x, x='accident_type', y='n', color='section', barmode='group',
        title='סוג תאונה בחלוקה למקטעים ותקופות', labels={'accident_type':'סוג תאונה', 'n': 'מספר תאונות'},
        category_orders={'section': ['arava_tihona_before_2012', 'arava_tihona_after_2012', 'arava_dromit_before_2012', 'arava_tihona_after_2012']}))
    # >> group_by(_.year)
    # >> summarize(killed=_.killed.sum(), lightly_injured=_.lightly_injured.sum())
)

fig.update_layout(
    legend=dict(
        yanchor='top',
        y=0.96,
        xanchor='right',
        x=0.99
    )
)
fig.show()

In [9]:
df.groupby('accident_type').sum()[casualty_cols].to_excel("acc_type.xlsx")

In [10]:

(
    df
    >> mutate(part=if_else(_.location <= 60.4, 'south', 'mid'),
        after=_.year >= 2012
    )
    >> mutate(part_time=_.part + if_else(_.after, '_after', '_before'))
    >> filter(_.part=='mid')
    # >> filter(_.location >= 60.4, _.location <= 180)
    >> group_by(_.part_time)
    >> count(_.accident_type)
    >> pipe(lambda x: px.bar(x, x='accident_type', y='n', color='part_time', barmode='group'))
    # >> group_by(_.year)
    # >> summarize(killed=_.killed.sum(), lightly_injured=_.lightly_injured.sum())
)

# Fatality of accidents

In [11]:
df.columns.tolist()

['year',
 'month',
 'day of week',
 'day/night',
 'accident_severity',
 'accident_type',
 'killed',
 'severly_injured',
 'lightly_injured',
 'injured_pedestrians',
 'casualties_ages_0-19',
 'casualties_ages_20-64',
 'casualties_ages_65_plus',
 'total_casualties',
 'vehicle_count',
 'drivers',
 'road_type',
 'localization_quality',
 'settlement',
 'road1',
 'location',
 'road2',
 'road3',
 'road4',
 'part',
 'after',
 'section']

In [17]:
df.groupby('section').accident_severity.value_counts(normalize=True).reset_index(name='severity')

,section,accident_severity,severity
0,arava_dromit_after_2012,קלה,0.660000
1,arava_dromit_after_2012,קשה,0.220000
2,arava_dromit_after_2012,קטלנית,0.120000
3,arava_dromit_before_2012,קלה,0.741573
4,arava_dromit_before_2012,קטלנית,0.157303
5,arava_dromit_before_2012,קשה,0.101124
6,arava_tihona_after_2012,קלה,0.392857
7,arava_tihona_after_2012,קשה,0.380952
8,arava_tihona_after_2012,קטלנית,0.226190
9,arava_tihona_before_2012,קלה,0.528000


In [25]:
(
    df.groupby(['year', 'part']).accident_severity.value_counts(normalize=True).reset_index(name='severity')
    >> filter(_.accident_severity=='קטלנית')
    >> pipe(lambda x: px.bar(x, x='year', y='severity', color='part', barmode='group'))
)